In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

df = DataFrame([{'product_id':23, 'name':'computer', 'wholesale_price': 500, 
                 'retail_price':1000, 'sales':100},
               {'product_id':96, 'name':'Python Workout', 'wholesale_price': 35,
                'retail_price':75, 'sales':1000},
               {'product_id':97, 'name':'Pandas Workout', 'wholesale_price': 35,
                'retail_price':75, 'sales':500},
               {'product_id':15, 'name':'banana', 'wholesale_price': 0.5,
                'retail_price':1, 'sales':200},
               {'product_id':87, 'name':'sandwich', 'wholesale_price': 3,
                'retail_price':5, 'sales':300},
               ])

df['current_net'] = ((df['retail_price'] - df['wholesale_price']) * df['sales'])
df

,product_id,name,wholesale_price,retail_price,sales,current_net
0,23,computer,500.0,1000,100,50000.0
1,96,Python Workout,35.0,75,1000,40000.0
2,97,Pandas Workout,35.0,75,500,20000.0
3,15,banana,0.5,1,200,100.0
4,87,sandwich,3.0,5,300,600.0


# Beyond 1

An alternative tax plan would charge 25% tax, but only on those products on which we would net more than 20,000. In such a case, how much would we make?

다른 방안 1, 순이익이 20,000을 초과하는 제품에 대해서만 25%의 세금을 부과하면 얼마를 벌게 될까 

In [2]:
# The short way, using lambda + the inline if-else
df['current_net'].apply(lambda c: c*0.75 if c > 20000 else c).sum()
#위에서 만든 current_net 컬럼의 값이 20000보다 크면 0.75를 곱하고, 그렇지 않으면 원래 값을 유지한 후, 그 합계를 구한다.
#lambda는 익명함수로, apply 메서드와 함께 사용되어 각 요소에 대해 조건부 연산을 수행한다.
#lambda함수 해석, 만약 c가 20000보다 크다면, c*0.75를 반환하고, 그렇지 않으면 c를 반환한다.
#원래는 하나의 값을 반환하는데, apply 메서드는 Series의 각 요소에 대해 이 함수를 적용하여 새로운 Series를 생성한다.
#마지막으로, sum() 메서드는 이 새로운 Series의 모든 값을 합산하여 최종 결과를 반환한다.


#.apply() 메서드는 DataFrame이나 Series의 각 요소에 대해 함수를 적용하는 데 사용된다.
#apply(함수) 이때 함수에 데이터프레임 컬럼 하나씩 인자로 전달되어 함수가 실행되고, 그 값을 반환함.
df['current_net'].apply(lambda c: c*0.75 if c > 20000 else c)

0    37500.0
1    30000.0
2    20000.0
3      100.0
4      600.0
Name: current_net, dtype: float64

In [3]:
# The longer way, defining a "real" function with a normal if-else
def calculate_tax(c):
    if c > 20000:
        return c * 0.75

    return c
df['current_net'].apply(calculate_tax).sum()
#lambda함수가 아닌, calculate_tax라는 이름의 일반 함수를 정의하여 동일한 작업을 수행한다.
#이 함수도 apply와 함께 사용되어 각 요소에 대해 조건부 연산을 수행한다.
#결과는 동일하게, current_net 컬럼의 값이 20000보다 크면 0.75를 곱하고, 그렇지 않으면 원래 값을 유지한 후, 그 합계를 구한다.

np.float64(88200.0)

# Beyond 2

Yet another alternative tax plan would charge 25% tax on products whose retail price is greater than 80, 10% tax on products whose retail price is between 30 and 80, and no tax on others. Implement and calculate the result of such a tax scheme.

다른 방안2, 소매가가 80보다 큰 제품에는 25% 세금을, 소매가가 30~80사이인 제품에는 10% 세금을, 그 외 제품은 세금을 부과하지 않음. 


In [4]:
# Use pd.cut to set the cutoffs, then translate from category to floats
df['after_tax'] = pd.cut(df['retail_price'], 
                   bins=[0, 30, 80, df['retail_price'].max()],
                   labels=[1, 0.9, 0.75]).astype(np.float64)

df['final_net'] = df['current_net'] * df['after_tax']
df

#after_tax라는 컬럼을 생성하는데, retail_price 컬럼의 값을 기준으로 구간을 나누고, 각 구간에 해당하는 세율을 적용한다.
#after_tax는 retail_price가 0에서 30 사이면 1, 30에서 80 사이면 0.9, 80 이상이면 0.75의 값을 갖는다.
#위의 방법대로 만든 after_tax 컬럼을 사용하여 final_net 컬럼을 생성한다.
df['final_net'].sum() #대체 방안1의 방법보다 더 많이 벌 수 있음.

np.float64(92200.0)

# Beyond 3

These long floating-point numbers are getting a bit hard to read. Set the `float_format` option in `pandas` such that the floating-point numbers will be displayed with commas every three digits before the decimal point, and only two digits after the decimal point. Note that this is a bit tricky, in that it requires understanding Python callables and the `str.format` method. 

부동소수점이 읽기 어려워지고 있다.
pandas의 float_format 옵션을 통해
소수점 앞의 숫자는 세 자리마다 쉼표로 구분하고, 소수점 아래는 소수점 둘째 자리까지만 표시되도록 해라.
이는 파이썬의 호출 가능한 객체와, str.format 메서드를 이해해야 하기에 다소 까다로움


In [5]:


# #pd.options.display.float_format 속성의 역할 확인
# test = pd.DataFrame({'val': [1234.5678, 9876543.21, 3.14159]})
# print("Before setting float_format:")
# print(test) #설정 전 출력 예시 1.234568e+03

# # 설정: 화면 출력 포맷만 바뀜(데이터 자체는 변경되지 않음)
# pd.options.display.float_format = '{:,.2f}'.format
# print("\nAfter setting float_format:")
# print(test) #설정 후 출력 예시 1.234.57

# # 리셋: 원래대로 돌아감
# pd.reset_option('display.float_format')
# print("\nAfter reset:")
# print(test)




pd.options.display.float_format = '{:,.2f}'.format

#callable 이란, 파이썬에서 호출할 수 있는 객체를 의미합니다.
# 파이썬에서 호출(call) 가능하다는 건, 뒤에 괄호 ( ) 를 붙여 실행할 수 있다는 뜻
#예로, 함수, 람다, __call__을 가진 객체, 메서드 등이 있습니다.
#pd.options.display.float_format이 작동하는 원리는
#pandas가 float_format에 전달된 callable에 float을 넣어 문자열을 얻음.
#str.format : 포맷 문자열 메서드, str에 '{:,.2f}'가 들어간 것. '{:,.2f}'는 서식 문자열로 
#숫자를 천 단위로 쉼표로 구분하고 소수점 이하 두 자리까지 표시하는 형식을 지정.
#: = 서식 지정의 시작을 나타냄 , = 천 단위 구분 기호 .2f = 소수점 이하 두 자리까지 표시하는 부동 소수점 숫자
# 파이썬의 문자열에는 .format() 메서드가 있어요.
# 즉, "asd".format(...) 라는 건 문자열 "asd" 안에서 중괄호 {}를 자리 표시자(placeholder) 로 쓰고, 그 자리에 .format()의 인자가 들어가도록 하는 거예요.
# float_format에 callable을 설정하면 표시(출력)만 바뀌고, 실제 데이터는 바뀌지 않음.



# pandas의 float_format 옵션은 “숫자를 문자열로 바꿔주는 호출 가능한 객체”를 받습니다.
# 그 중 하나가 바로 '{:,.2f}'.format 이예요.
# 이걸 그대로 pd.set_option("display.float_format", '{:,.2f}'.format) 에 넘기면, 판다스가 숫자를 출력할 때마다 이 함수를 호출해주게 되는 거예요 ✅



# pd.options.display.float_format에 넣는 값은 호출 가능한(callable) 객체
# 따라서 '{:,.2f}' 이라는 문자열 자체를 넣는 게 아니라,
# '{:,.2f}'.format 이라는 호출 가능한 객체를 넣어야 함.


# pd.options.display.float_format = '{:,.2f}'.format은
# **“숫자 → 문자열 변환 규칙을 가진 함수”**를 pandas에게 넘겨주는 거고,
# # 이건 사실상 함수를 하나 정의한 것과 같은 효과예요.
# def add(a,b):
#     return a + b    
# a = add
# a(1,2)
#여기서 a에 add라는 함수를 할당한 것처럼 pd.options.display.float_format에 '{:,.2f}'.format을 할당한 것.


# '{:,.2f}'.format은 문자열 메소드이지만, Python에서는 이것도 함수 객체라서
# pd.options.display.float_format에 그대로 할당할 수 있어요.
# 결국 “숫자를 받아서 문자열로 변환하는 함수”를 Pandas에 넘겨준 것.

In [6]:
df

,product_id,name,wholesale_price,retail_price,sales,current_net,after_tax,final_net
0,23,computer,500.00,1000,100,"50,000.00",0.75,"37,500.00"
1,96,Python Workout,35.00,75,1000,"40,000.00",0.90,"36,000.00"
2,97,Pandas Workout,35.00,75,500,"20,000.00",0.90,"18,000.00"
3,15,banana,0.50,1,200,100.00,1.00,100.00
4,87,sandwich,3.00,5,300,600.00,1.00,600.00
